## 1. Imports and loading data

In [18]:
import pandas as pd
import numpy as np
import sklearn
import random
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA as sklearn_pca

np.set_printoptions(formatter={'float':'{:0.4f}'.format})
pd.set_option('display.precision', 5)

filePath = "C:/Chang/NEU/courses/cs6220 datamining/project/dataset_diabetes/diabetic_data.csv"
# Read dataset using pandas
data = pd.read_csv(filePath, sep=',',header=0)

## 2. Preprocessing
This data set contains both numeric and nominal data types. For numeric data we need to do normalization, while for categorical data, we perform one-hot encoding.

In [19]:
# Drop encounter_id and patient_nbr. Drop weight and payer_code which has 97% missing data
df = data.drop(['encounter_id', 'patient_nbr', 'weight', 'payer_code'], axis = 1)

In [20]:
# TODO: Currently performed binary encoding. Should try data binning.
df['diag_1'] = np.where(df['diag_1'].str.contains('250'), 1, 0)
df['diag_2'] = np.where(df['diag_2'].str.contains('250'), 1, 0)
df['diag_3'] = np.where(df['diag_3'].str.contains('250'), 1, 0)

# Encode readmitted into 0 or 1
df['readmitted'] = np.where(df['readmitted'].str.contains('<30'), 1, 0)

In [21]:
# Standardizing numeric data
from sklearn.preprocessing import StandardScaler
numData = df[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 
              'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']]
numData_std = StandardScaler().fit_transform(numData)

In [22]:
# Confirm that the numeric data are standardized
df_numData = pd.DataFrame(numData_std)
df_numData.describe()

,0,1,2,3,4,5,6,7
count,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05
mean,2.35731e-14,3.24394e-16,4.23327e-15,-2.18372e-14,1.00082e-13,2.50927e-14,-4.31450e-14,1.28417e-13
std,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00
min,-1.13765e+00,-2.13963e+00,-7.85398e-01,-1.84827e+00,-2.91461e-01,-2.12620e-01,-5.03276e-01,-3.32160e+00
25%,-8.02651e-01,-6.14795e-01,-7.85398e-01,-7.40920e-01,-2.91461e-01,-2.12620e-01,-5.03276e-01,-7.35733e-01
50%,-1.32655e-01,4.59666e-02,-1.99162e-01,-1.25726e-01,-2.91461e-01,-2.12620e-01,-5.03276e-01,2.98612e-01
75%,5.37341e-01,7.06728e-01,3.87074e-01,4.89467e-01,-2.91461e-01,-2.12620e-01,2.88579e-01,8.15784e-01
max,3.21732e+00,4.51881e+00,2.73202e+00,7.99483e+00,3.28509e+01,8.14667e+01,1.61257e+01,4.43599e+00


In [23]:
df[['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 
              'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']] = df_numData

In [24]:
# Apply one-hot encoding to categorical columns
oneHotData = pd.get_dummies(df, columns=['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id',
                                         'admission_source_id', 'medical_specialty', 
                                         'diag_1', 
                                         'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 
                                         'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 
                                         'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 
                                         'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 
                                         'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
                                         'glipizide-metformin', 'glimepiride-pioglitazone', 
                                         'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 
                                         'diabetesMed', 'readmitted'],
                            prefix=['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 
                                    'admission_source_id', 'medical_specialty', 
                                    'diag_1', 'diag_2', 'diag_3', 
                                    'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 
                                    'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 
                                    'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 
                                    'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin',
                                    'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 
                                    'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed',
                                    'readmitted'])
oneHotData.describe()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_?,race_AfricanAmerican,...,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,readmitted_0,readmitted_1
count,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,1.01766e+05,101766.00000,101766.00000,...,101766.00000,101766.00000,101766.00000,1.01766e+05,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000,101766.00000
mean,2.35731e-14,3.24394e-16,4.23327e-15,-2.18372e-14,1.00082e-13,2.50927e-14,-4.31450e-14,1.28417e-13,0.02234,0.18877,...,0.99998,0.00002,0.99999,9.82646e-06,0.46195,0.53805,0.22997,0.77003,0.88840,0.11160
std,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,1.00000e+00,0.14777,0.39132,...,0.00443,0.00443,0.00313,3.13472e-03,0.49855,0.49855,0.42081,0.42081,0.31487,0.31487
min,-1.13765e+00,-2.13963e+00,-7.85398e-01,-1.84827e+00,-2.91461e-01,-2.12620e-01,-5.03276e-01,-3.32160e+00,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000e+00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,-8.02651e-01,-6.14795e-01,-7.85398e-01,-7.40920e-01,-2.91461e-01,-2.12620e-01,-5.03276e-01,-7.35733e-01,0.00000,0.00000,...,1.00000,0.00000,1.00000,0.00000e+00,0.00000,0.00000,0.00000,1.00000,1.00000,0.00000
50%,-1.32655e-01,4.59666e-02,-1.99162e-01,-1.25726e-01,-2.91461e-01,-2.12620e-01,-5.03276e-01,2.98612e-01,0.00000,0.00000,...,1.00000,0.00000,1.00000,0.00000e+00,0.00000,1.00000,0.00000,1.00000,1.00000,0.00000
75%,5.37341e-01,7.06728e-01,3.87074e-01,4.89467e-01,-2.91461e-01,-2.12620e-01,2.88579e-01,8.15784e-01,0.00000,0.00000,...,1.00000,0.00000,1.00000,0.00000e+00,1.00000,1.00000,0.00000,1.00000,1.00000,0.00000
max,3.21732e+00,4.51881e+00,2.73202e+00,7.99483e+00,3.28509e+01,8.14667e+01,1.61257e+01,4.43599e+00,1.00000,1.00000,...,1.00000,1.00000,1.00000,1.00000e+00,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


## 3. Train Logistic Regression Model to predict readmission

In [25]:
# prepare train and test data for logistic regression
# Group data according to readmitted or not
readmitted_pos = oneHotData[oneHotData['readmitted_1'] == 1]
readmitted_neg = oneHotData[oneHotData['readmitted_0'] == 1]
readmitted_pos.shape

(11357, 242)

In [124]:
# Dataframe to list
readmitted_pos_list = readmitted_pos.values.tolist()
readmitted_neg_list = readmitted_neg.values.tolist()

# Randomly select negative data to have same size as pos data
sample_neg = random.sample(readmitted_neg_list, 12000)

# Prepare train and test data
train = readmitted_pos_list[0:8500] + sample_neg[0:8500]
test = readmitted_pos_list[8500:] + sample_neg[8500:]

# The last column is 'readmitted' label
train_X = []
train_Y = []
for x in train:
    train_X.append(x[:-2])
    train_Y.append([x[-1]])

test_X = []
test_Y = []
for x in test:
    test_X.append(x[:-2])
    test_Y.append([x[-1]])

In [125]:
# train logesitc regression model
lr = sklearn.linear_model.LogisticRegression()

lr.fit(train_X, train_Y)

predicts = lr.predict(test_X)

count = 0
for p, y in zip(predicts, test_Y):
    if p == y:
        count += 1
print(count/len(test_Y))

C:\Users\lay42\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5656756331602958


The accuracy of the model is 56.57%, next we want to reduce the dimensionality of the training data using PCA and retrain the model to see if we can improve the performance.

## 4. PCA

In [126]:
pca = sklearn_pca(n_components=6)
data = pca.fit_transform(train_X)
data.shape

(17000, 6)

In [127]:
test_X = pca.transform(test_X)
test_X.shape

(6357, 6)

In [128]:
print(pca.explained_variance_ratio_)

[0.1256 0.1111 0.0678 0.0630 0.0559 0.0522]


In [129]:
# Retrain the model
lr.fit(data, train_Y)
predicts = lr.predict(test_X)

count = 0
for p, y in zip(predicts, test_Y):
    if p == y:
        count += 1
print(count/len(test_Y))

0.6064181217555451


C:\Users\lay42\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy of the model has been improved to 60.64% after training with low-dimensional data.